In [1]:
import requests

API_URL = "http://localhost:5433"
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"

def generate(prompt: str) -> str:
    """เรียก /api/generate เพื่อสร้างข้อความตอบกลับ"""
    res = requests.post(
        f"{API_URL}/api/generate",
        json={
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False
        }
    )
    res.raise_for_status()
    return res.json()["response"]

if __name__ == "__main__":
    print(generate("สวัสดี"))

สวัสดีครับ! ยินดีที่ได้รู้จักครับ ผมชื่อ Typhoon สร้างโดย SCB 10X เพื่อช่วยเหลือคุณอย่างมีประโยชน์ ปลอดภัย และซื่อสัตย์

มีอะไรให้ผมช่วยวันนี้ครับ?


In [61]:
import os

def load_and_chunk(directory: str, chunk_size: int = 800):
    """อ่านไฟล์ .txt ในโฟลเดอร์ แล้วแบ่งเป็นชิ้นละ chunk_size คำ"""
    chunks = []
    for fname in os.listdir(directory):
        if not fname.endswith(".txt"):
            continue
        text = open(os.path.join(directory, fname), encoding="utf-8").read()
        words = text.split()
        for i in range(0, len(words), chunk_size):
            chunk = " ".join(words[i : i + chunk_size])
            # เก็บ metadata เบื้องต้นด้วย
            chunks.append({
                "content": chunk,
                "source": fname
            })
    return chunks

# usage
directory = "Raw-data-from-TTT"
list_of_chunks = load_and_chunk(directory)
print(f"ได้ทั้งหมด {len(list_of_chunks)} chunks")


ได้ทั้งหมด 14 chunks


In [73]:
from sentence_transformers import SentenceTransformer

# โหลดโมเดล embedding
embed_model = SentenceTransformer("BAAI/bge-m3")

d:\TTT_Trainee\Project_1\AI-Chatbot\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Arther\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [74]:
# แปลงเป็นเวกเตอร์ 768 มิติ
texts = [c["content"] for c in list_of_chunks]
embeddings = embed_model.encode(texts, show_progress_bar=True)

# เพิ่ม embedding ลงใน list_of_chunks
for chunk, emb in zip(list_of_chunks, embeddings):
    chunk["vector"] = emb.tolist()

Batches: 100%|██████████| 1/1 [00:16<00:00, 16.80s/it]


In [76]:
import psycopg2

conn = psycopg2.connect(
    dbname="mydb", user="admin", password="1234",
    host="localhost", port="5432"
)
cur = conn.cursor()
# สร้างตารางถ้ายังไม่มี
cur.execute("""
    CREATE TABLE IF NOT EXISTS documents (
        id SERIAL PRIMARY KEY,
        content TEXT,
        source TEXT,
        embedding VECTOR(1024)
    )
""")
conn.commit()

# แทรกข้อมูล
for c in list_of_chunks:
    cur.execute("""
        INSERT INTO documents (content, source, embedding)
        VALUES (%s, %s, %s)
    """, (c["content"], c["source"], c["vector"]))
conn.commit()
cur.close()
conn.close()


In [ ]:
import psycopg2

# 2. ฟังก์ชันสืบค้นเอกสารจาก Postgres ตามความใกล้เคียงของเวกเตอร์
def retrieve_similar(query: str, top_k: int = 6):
    """
    รับข้อความค้นหา (query) -> แปลงเป็นเวกเตอร์ -> 
    SELECT เอกสารที่ embedding ใกล้เคียงที่สุด top_k ชิ้น
    """
    # แปลง query เป็นเวกเตอร์
    q_vec = embed_model.encode([query], show_progress_bar=False)[0].tolist()

    # เชื่อมต่อฐานข้อมูล
    conn = psycopg2.connect(
        dbname="mydb", user="admin", password="1234",
        host="localhost", port="5432"
    )
    cur = conn.cursor()

    # ใช้ operator <-> ของ pgvector เพื่อคำนวณระยะ Euclidean
    # ต้องแปลง array เป็น vector ด้วย ::vector
    cur.execute("""
        SELECT content, source
        FROM documents
        ORDER BY embedding <-> (%s::vector)
        LIMIT %s
    """, (q_vec, top_k))

    rows = cur.fetchall()
    cur.close()
    conn.close()

    # คืนผลเป็น list ของ dict
    return [{"content": r[0], "source": r[1]} for r in rows]


# 3. ฟังก์ชันรวม context และเรียก LLM (ตัวอย่างสำหรับ local API)
import requests

API_URL = "http://localhost:5433"       # แก้เป็น URL ของ LLM service
MODEL_NAME = "scb10x/typhoon2.1-gemma3-4b:latest"

def generate_rag_response(query: str, top_k: int = 6) -> str:
    # 3.1 สืบค้นเอกสารที่เกี่ยวข้อง
    docs = retrieve_similar(query, top_k)

    # 3.2 ประกอบ context
    context = "\n\n".join(
        f"Source: {doc['source']}\n{doc['content']}" for doc in docs
    )
    prompt = (
    "คุณคือผู้ช่วยอัจฉริยะชื่อ TTT-Assistant จากบริษัท ทีทีที บราเธอร์ส จำกัด "
    "กรุณาตอบคำถามให้ละเอียดและครบถ้วนที่สุด โดยใช้ข้อมูลจาก context ที่มี\n"
        f"---\nContext: {context}\n---"
        "\n"
        f"Question: {query}"
        "\nAnswer:"
    )

    # 3.3 เรียก LLM API เพื่อสร้างคำตอบ
    res = requests.post(
        f"{API_URL}/api/generate",
        json={
            "model": MODEL_NAME,
            "prompt": prompt,
            "stream": False
        }
    )
    res.raise_for_status()
    return res.json()["response"]

# 4. วิธีใช้งาน
if __name__ == "__main__":
    question = "กฏการลางานมีอะไรบ้าง"
    answer = generate_rag_response(question, top_k=3)
    print(answer)

จากข้อมูลที่ให้มา กฏการลางานมีดังนี้:

*   **ลาพักร้อน:** ได้ 30 วัน/ปี
*   **ลาป่วย:** แจ้งภายในเดือนนั้น และหากป่วย ≥ 2 วัน ต้องมีใบรับรองแพทย์
*   **ลาล่วงหน้า:** ลากิจ - แจ้งลาล่วงหน้าอย่างน้อย 3 วัน (ผ่านหัวหน้า + HR)
*   **ลา ≥ 2 วัน:** ต้องแจ้งล่วงหน้า 1 สัปดาห์
*   **แลก Point:** สามารถแลกเป็น Point ได้ (200 Point/วัน ภายใน 30 พ.ย.)
